In [0]:
##Convert raw csv files to delta format

# ----------------------------------------
# 1. Read CSV into DataFrame
# ----------------------------------------
events = (
    spark.read
         .format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load("/Volumes/workspace/ecommerce/ecommerce_data/2019-*.csv")
)

# ----------------------------------------
# 2. avoid duplicates in case if any
# ----------------------------------------
events_all = events.union(events)
events_all = events_all.dropDuplicates(["product_id"])


# ----------------------------------------
# 2. Register table in metastore
# ----------------------------------------
events_all.write \
      .format("delta") \
      .mode("overwrite") \
      .saveAsTable("workspace.ecommerce.events_delta")

# ----------------------------------------
# 4. Simulate incremental small appends
# ----------------------------------------
small_file_threshold = 200000

if events.count() < small_file_threshold:
    for i in range(3):
        events.limit(500).write \
              .format("delta") \
              .mode("append") \
              .saveAsTable("workspace.ecommerce.events_delta")
    print("Small-file simulation complete")
else:
    print("Skipping small-file simulation for large CSVs")

# ----------------------------------------
# 5. Optimize (compact small files)
# ----------------------------------------
spark.sql("OPTIMIZE workspace.ecommerce.events_delta")

# ----------------------------------------
# 6. Optional: Clean old files (after retention period)
# ----------------------------------------
# spark.sql("VACUUM workspace.ecommerce.events_delta RETAIN 168 HOURS")


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:724)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can